In [ ]:
# default_exp inference_stl

# Inference for single-task learning

In [ ]:
#hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#export

import numpy as np
from fastai.basics import *
from vqa.utils import most_common

In [ ]:
#export

def get_test_inferences(dls, learn, tst_df):
    tst_dl = dls.test_dl(tst_df)
    tst_learn = Learner(
        dls,
        learn.model,
        loss_func=learn.loss_func,
        splitter=learn.model.splitter
    )
    probs, targets, preds = tst_learn.get_preds(dl=tst_dl, with_decoded=True)
    
    inference_df = tst_df.copy()
    inference_df['preds'] = np.array(dls.vocab)[preds.cpu().numpy()]
    inference_df = inference_df.groupby(by='video_name').agg({
        'label': first,
        'preds': list,
    })
    inference_df['pred'] = inference_df['preds'].apply(most_common)
    return inference_df

def make_submission_preds(preds):
    return [','.join(sorted(ds[1:] for ds in pred.split(','))) for pred in preds]


In [ ]:
#hide
test_eq(make_submission_preds(["D1_1,D5_4"]), ['1_1,5_4'])
test_eq(make_submission_preds(["D3_2,D8_1,D5_4"]), ['3_2,5_4,8_1'])
test_eq(make_submission_preds(["D2_3"]), ['2_3'])